# VacationPy 

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map. 

### Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.
Drop any rows will null values.

### Hotel Map 
Store into vairable named hotel_df
Add a "hotel name" column to this dataframe
Set parameter to search for hotels within 5000 m
Communicate with google API to get city coordinates
Store the first hotel result into city DataFrame
Plot the markers on top of the heatmap